# 데이터 세트 준비

In [ ]:
# 데이터세트 다운로드
!wget http://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar

--2024-12-20 00:30:48--  http://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar
pjreddie.com (pjreddie.com) 해석 중... 162.0.215.52
다음으로 연결 중: pjreddie.com (pjreddie.com)|162.0.215.52|:80... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 301 Moved Permanently
위치: https://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar [따라감]
--2024-12-20 00:30:49--  https://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar
다음으로 연결 중: pjreddie.com (pjreddie.com)|162.0.215.52|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 200 OK
길이: 1999639040 (1.9G) [application/x-tar]
저장 위치: `VOCtrainval_11-May-2012.tar'

VOCtrainval_11-May-  26%[====>               ] 500.16M   782KB/s    /  5m 31s  

2024-12-20 00:37:25 (1.51 MB/s) - 524452033 바이트에 연결 끊음. 다시 시도 중.

--2024-12-20 00:37:26--  (시도횟수: 2)  https://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar
다음으로 연결 중: pjreddie.com (pjreddie.com)|162.0.215.52|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 200 OK
길이: 1999639040 (1.9G) [application/x-tar]
저장 위치: `VOCtrainv

In [ ]:
# 압축 해제
!tar -xf VOCtrainval_11-May-2012.tar
# 원본 압축 파일 제거
!rm VOCtrainval_22-May-2012.tar

In [ ]:
# 클래스 리스트 파일 생성
classes = ["aeroplane\n", "bicycle\n", "bird\n", "boat\n", "bottle\n", 
       "bus\n", "car\n", "cat\n", "chair\n", "cow\n", "diningtable\n", 
       "dog\n", "horse\n", "motorbike\n", "person\n", "pottedplant\n", 
       "sheep\n", "sofa\n", "train\n", "tvmonitor"]
with open('vocnames.txt','w') as f: # txt파일 생성
       f.writelines(classes)

In [ ]:
# labels 폴더 생성
!git clone https://github.com/ssaru/convert2Yolo.git
# 파일 포맷 변환 xml to txt
!cd convert2Yolo && python3 example.py --datasets VOC \
--img_path ../VOCdevkit/VOC2012/JPEGImages/ \
--label ../VOCdevkit/VOC2012/Annotations/ \
--convert_output_path ../VOCdevkit/VOC2012/JPEGImages/ \
--img_type ".jpg" \
--manifest_path ../ \
--cls_list_file ../vocnames.txt## 폴더 경로 : ../VOCdevkit/VOC2012/JPEGImages/
## * yolov5는 훈련할 때 이미지 파일과 라벨링된 텍스트 파일의 1)경로 와 2)이름은 동일해야한다.

In [ ]:
# 데이터 분할 
## 훈련 : 검증 = 7 : 3
import os 
import shutil
from tqdm import tqdm

## 폴더 경로 지정
data_root='./VOCData'
val_root=os.path.join(data_root,'val')
train_root=os.path.join(data_root,'train')
## 폴더 생성
os.makedirs(val_root,exist_ok=True)
os.makedirs(train_root,exist_ok=True)
## 이미지 경로 읽어오기
with open('./manifest.txt') as f:
    files=f.readlines()
    
for idx, img_path in tqdm(enumerate(files)):
    img_src=img_path.split('₩n')[0]
    txt_src=os.path.splitext(img_src)[0]+'.txt'
    img_name=os.path.split(img_src)[-1]
    text_name=os.path.split(txt_src)[-1]
    print(img_src)
    print(txt_src)
    print(img_name)
    print(text_name)
    # if idx%10<3:
    #     img_dst=os.path.join(val_root,img_name)
    #     text_dst=os.path.join(val_root,text_name)
    # else:
    #     img_dst=os.path.join(train_root,img_name)
    #     text_dst=os.path.join(train_root,text_name)
    # shutil.copy2(img_src,img_dst)
    # shutil.copy2(txt_src,text_dst)


# Yolov5 훈련

In [ ]:
# 훈련환경 준비
!git clone -b v7.0 https://github.com/jetsonai/yolov5 # 깃 저장소 복사
%cd yolov5 #경로 이동
%pip install -qr requirements.txt # 필수(requirements)패키지 설치

In [ ]:
# 훈련 환경 테스트 v1
python detect.py --source ../DeepLearning4Projects/Chap7/data/crosswalk_cctv_01.mp4

In [ ]:
# 훈련 환경 테스트 v2
## 입력 데이터 해상도 640*640, confidence >=0.25인 객체만 검출
!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source data/images
import cv2
import matplotlib.pyplot as plt
cv2.imshow(cv2.imread('runs/detect/exp/zidane.jpg'))

In [ ]:
# yaml 파일 제작
text_lines='''
train:
    - ./VOCData/train
val:
    - ./VOCData/val
    
# Classes
names:
    0: aeroplane                  
    1: bicycle                    
    2: bird                       
    3: boat                       
    4: bottle                     
    5: bus                        
    6: car                        
    7: cat                        
    8: chair                      
    9: cow                        
    10: diningtable               
    11: dog                       
    12: horse                     
    13: motorbike                 
    14: person                    
    15: pottedplant               
    16: sheep                     
    17: sofa                      
    18: train                     
    19: tvmonitor
'''

with open('./yolov5/vocdata.yaml','w') as f:
    f.write(text_lines)

In [ ]:
# 훈련
!python train.py --img 480 --batch 16 --epochs 20 --data vocdata.yaml --weights yolov5s.pt --cache
## 훈련 해상도 : 640*640 => 480*480
## 배치 크기 : 16, 에포크 수 : 20, 가중치 : yolov5s.pt
## cache : 훈련을 빠르게 하기 위해 사용. 데이터세트를 읽은 후 이진 파일로 저장

In [ ]:
# 추론 테스트
cd ./yolov5
python detect.py --weights ./best.pt --img 640 --conf 0.25 --source ../data/crosswalk_cctv_01.mp4 --view-img